In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy
import scipy.stats

from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger
from textblob import Word
from textblob.base import BaseTokenizer
from textblob import TextBlob
from textblob import Blobber
from textblob.tokenizers import WordTokenizer

from numpy import isnan
from nltk.corpus import stopwords
data=pd.read_csv("Assignment 3 Edmunds Posts.csv", usecols=[0])

data['Posts'] = data['Posts'].map(lambda x: x.replace("LexusES","ES"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("ES330","ES"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("LS460","LS"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("LS470","LS"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("LexusLS","LS"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("LexusRX","RX"))

sw=stopwords.words('english')

sw.remove('not')

models_skinny=["es","ls","rx","a8","a6","3series","5series","7series","xj","sclass"] 

def tokenizer (xstring):
    global sw
    tokens=list(TextBlob(str(xstring)).words)
    removeStopWords=[word for word in tokens if word.lower() not in sw]
    lemmaed=[Word(w).lemmatize() for w in removeStopWords]
    lowercase=[word.lower() for word in lemmaed]
    return lowercase

def finder (tokensList, modelList, numberWords):
    blanklist=[]
    for i in xrange(len(tokensList)):
        if tokensList[i] in modelList:
            blanklist.append(tokensList[i-numberWords:i+1+numberWords])
    return blanklist
    
def compiler(postseries,modelsList,numberofwords):
    tb=Blobber(tokenizer=WordTokenizer())
    
    newseries=postseries.apply(tokenizer)
    newseries=newseries.apply(lambda x: finder(x,modelsList, numberofwords))
    newseries=newseries.apply(lambda x: [' '.join(inner) for inner in x])
    
    models=[" "+x+" " for x in modelsList]
    df=pd.DataFrame(columns=modelsList).join(newseries, how="outer")
    
    for index,l in enumerate(newseries):
        for string in l: 
                for model, model_skinny in zip(models,modelsList): #loop over all the models in model list
                    if model in string: #check if model is in a particular list
                        if isnan(df[model_skinny].iloc[index]): #correcting for neutral
                            df[model_skinny].iloc[index]=0
                        df[model_skinny].iloc[index]+=tb(string).sentiment[0]  
    return df
    
    


rsentiment1 = compiler(data['Posts'],models_skinny,2)
rsentiment = rsentiment1.drop(['Posts'], axis = 1)

In [ ]:
testing = ['ES', 'LS', 'RX', 'A8', 'A6', '3series', '5series', '7series', 'XJ', 'Sclass']
rsentiment.columns = testing 
rsentiment = rsentiment * 5


#names = ['ES-LS','ES-RX','ES-A8','ES-A6','ES-3series','ES-5series','ES-7series','ES-XJ','ES-Sclass','LS-RX','LS-A8','LS-A6','LS-3series','LS-5series','LS-7series','LS-XJ','LS-Sclass','RX-A8','RX-A6','RX-3series','RX-5series','RX-7series','RX-XJ','RX-Sclass','A8-A6','A8-3series','A8-5series','A8-7series','A8-XJ','A8-Sclass','A6-3series','A6-5series','A6-7series','A6-XJ','A6-Sclass','3series-5series','3series-7series','3series-XJ','3series-Sclass','5series-7series','5series-XJ','5series-Sclass','7series-XJ','7series-Sclass','XJ-Sclass']
names = list(rsentiment.columns)
#i=1
#j=1

##Create the sentiment difference dataframe (ES-LS, ES-RX, etc)
length_senti = len(rsentiment)
width_senti = len(rsentiment.T)
comparisons = pd.DataFrame(index=np.array(range(length_senti)))
#Vectors for future use in the nodes
node_name_1 = []
node_name_2 = []

n=1 #For counter
for i in range(width_senti-1):
    for j in range(width_senti - n):
        comparisons[str(names[i]) + '-' + str(names[(j+n)])] = np.array(rsentiment[[i]]) - np.array(rsentiment[[j+n]])
        node_name_1.append(names[i])
        node_name_2.append(names[(j+n)])
    n=n+1

length_comp = len(comparisons.T)

averages_df = pd.DataFrame(index=comparisons.columns, columns = ['Positive','Negative'])



for i in comparisons.columns:
    pos_sum = 0
    neg_sum = 0    
    pos_sum = [pos_sum + x for x in comparisons[i] if x > 0]
    neg_sum = [neg_sum + x for x in comparisons[i] if x < 0]
    if len(pos_sum) != 0:    
        averages_df.loc[i,'Positive'] = sum(pos_sum)/len(pos_sum)
    if len(neg_sum) != 0:
        averages_df.loc[i,'Negative'] = sum(neg_sum)/len(neg_sum)

averages_df['Node 1'] = node_name_1
averages_df['Node 2'] = node_name_2

##Creating the network graph
G=nx.DiGraph()

#Creating the labels for the nodes and adding a node to the G object
labels = {}
opp_labels = {}
for i, value in enumerate(names):
    labels[i] = str(value)
    opp_labels[value] = i
    G.add_node(i) #Adding the nodes
    

#Create the list of tuples for edges
node_list = []
for index, pref in enumerate(averages_df.loc[:,'Positive']): #Positives
    if ~np.isnan(pref):
        node_list.append((pd.Series(averages_df.loc[:,'Node 2'])[index],pd.Series(averages_df.loc[:,'Node 1'])[index]))
        G.add_edge(*(opp_labels[pd.Series(averages_df.loc[:,'Node 2'])[index]],opp_labels[pd.Series(averages_df.loc[:,'Node 1'])[index]]))
for index, pref in enumerate(averages_df.loc[:,'Negative']): #Negatives
    if ~np.isnan(pref):
        node_list.append((pd.Series(averages_df.loc[:,'Node 1'])[index],pd.Series(averages_df.loc[:,'Node 2'])[index]))
        G.add_edge(*(opp_labels[pd.Series(averages_df.loc[:,'Node 1'])[index]],opp_labels[pd.Series(averages_df.loc[:,'Node 2'])[index]]))


pos = nx.circular_layout(G) #Positions

#nx.edges([(1,2),(1,0)])#node_list)

nx.draw_networkx_labels(G,pos, labels, font_size=16)
#nx.draw_networkx_nodes(G,pos,node_color='skyblue')
nx.draw(G,pos,node_size=1000)
###nx.Graph(node_list)

fig1 = plt.gcf()
plt.figure(num=None, figsize=(30,20))
#plt.savefig('network.png', dpi=100)

plt.show()
fig1.savefig('network.png', dpi=1000)


##Coding pagerank ------- Weighted


#Weights out
wout_dict = {}
for index, value in enumerate(averages_df.ix[:,'Positive']):
    if ~np.isnan(value):
        if averages_df.ix[index,'Node 2'] in wout_dict:
            wout_dict[averages_df.ix[index,'Node 2']] = wout_dict[averages_df.ix[index,'Node 2']] + 1
        else:
            wout_dict[averages_df.ix[index,'Node 2']] = 1
for index, value in enumerate(averages_df.ix[:,'Negative']):
    if ~np.isnan(value):
        if averages_df.ix[index,'Node 1'] in wout_dict:
            wout_dict[averages_df.ix[index,'Node 1']] = wout_dict[averages_df.ix[index,'Node 1']] + 1
        else:
            wout_dict[averages_df.ix[index,'Node 1']] = 1

#Weights in
win_dict = {}
for index, value in enumerate(averages_df.ix[:,'Positive']):
    if ~np.isnan(value):
        if averages_df.ix[index,'Node 1'] in win_dict:
            win_dict[averages_df.ix[index,'Node 1']] = win_dict[averages_df.ix[index,'Node 1']] + 1
        else:
            win_dict[averages_df.ix[index,'Node 1']] = 1
for index, value in enumerate(averages_df.ix[:,'Negative']):
    if ~np.isnan(value):
        if averages_df.ix[index,'Node 2'] in win_dict:
            win_dict[averages_df.ix[index,'Node 2']] = win_dict[averages_df.ix[index,'Node 2']] + 1
        else:
            win_dict[averages_df.ix[index,'Node 2']] = 1

#Counting the number of outputs for each node
counts_dict = {}
#Create a dict of price data
price_dict = {'A6':20000, 'A8':12000, '3series':220000, '5series':60000, '7series':14000, 'XJ':6600, 'ES':135000, 'LS':30000, 'RX':120000, 'Sclass':25000}


price_list = pd.Series(name='Price')
for dictkey in price_dict:
    price_list[dictkey] = price_dict[dictkey]

for index, value in enumerate(averages_df.ix[:,'Negative']):
    if np.isnan(value):
        continue
    else:
        if averages_df.ix[index,'Node 1'] in counts_dict:
            counts_dict[averages_df.ix[index,'Node 1']] += 1
        else:
            counts_dict[averages_df.ix[index,'Node 1']] = 1


for index, value in enumerate(averages_df.ix[:,'Positive']):
    if np.isnan(value):
        continue
    else:
        if averages_df.ix[index,'Node 2'] in counts_dict:
            counts_dict[averages_df.ix[index,'Node 2']] += 1
        else:
            counts_dict[averages_df.ix[index,'Node 2']] = 1


Remaining Tasks

Creating the matrix
weighting the values
Find Eigenvector/PageRank
Coding Unweighted pagerank
Counting the number of outputs for each node
Creating the matrix
Find Eigenvector/PageRank
Indexing

In [ ]:
Matrix = pd.DataFrame(index=names, columns=names)

for index, value in enumerate(averages_df.ix[:,'Positive']):
    if ~np.isnan(value):
        Matrix.ix[averages_df.ix[index,'Node 1'], averages_df.ix[index,'Node 2']] = ((wout_dict[averages_df.ix[index,'Node 2']])*win_dict[averages_df.ix[index,'Node 2']]*value)
for index, value in enumerate(averages_df.ix[:,'Negative']):
    if ~np.isnan(value):
        Matrix.ix[averages_df.ix[index,'Node 2'], averages_df.ix[index,'Node 1']] = ((wout_dict[averages_df.ix[index,'Node 1']])*win_dict[averages_df.ix[index,'Node 1']]*abs(value))

#weighting the values
vector_scalar = []
for row in range(len(Matrix)):
    setofmakes = []
    for make, value in enumerate(Matrix.ix[row,:]):
        if ~np.isnan(value):
            setofmakes.append(Matrix.index[make])
    temp_win = 0
    temp_wout = 0
    for i in setofmakes:
        temp_win = win_dict[i] + temp_win
        temp_wout = wout_dict[i] + temp_wout
    vector_scalar.append((1.0/(temp_wout*temp_win)))

Matrix = (Matrix.T * vector_scalar).T


Matrix2 = pd.DataFrame([1,1,1,1,1,1,1,1,1,1])

Matrix.fillna(0, inplace=True)
new_names = list(Matrix.index.values)
Matrix = np.matrix(Matrix) #Convert to Matrix
Matrix2= np.matrix(Matrix2) #Convert to Matrix


In [ ]:
#Find Eigenvector/PageRank
iterations = 25

for i in range(iterations):
    EV = Matrix*Matrix2
    EV= EV/np.linalg.norm(EV)
    if np.array_equal(np.array(Matrix2),np.array(EV)):
        print 'Convergence after ', i
        break
Matrix2 = new


In [ ]:
newseries = pd.DataFrame(new, index=new_names)

#Assign a index name to the pagerank
final = pd.concat([newseries,price_list], axis=1)
final.columns = ['PageRank','Sales'] 
print('Weighted Correlation')
print(scipy.stats.spearmanr(final['PageRank'], final['Sales']))
#print(scipy.stats.pearsonr(final['PageRank'], final['Price']))


In [ ]:

##Coding pagerank ----- Unweighted

#Counting the number of outputs for each node
counts_dict = {}

for index, value in enumerate(averages_df.ix[:,'Negative']):
    if np.isnan(value):
        continue
    else:
        if averages_df.ix[index,'Node 1'] in counts_dict:
            counts_dict[averages_df.ix[index,'Node 1']] += 1
        else:
            counts_dict[averages_df.ix[index,'Node 1']] = 1


for index, value in enumerate(averages_df.ix[:,'Positive']):
    if np.isnan(value):
        continue
    else:
        if averages_df.ix[index,'Node 2'] in counts_dict:
            counts_dict[averages_df.ix[index,'Node 2']] += 1
        else:
            counts_dict[averages_df.ix[index,'Node 2']] = 1


#Creating the matrix
A = pd.DataFrame(index=names, columns=names)

for index, value in enumerate(averages_df.ix[:,'Positive']):
    if ~np.isnan(value):
        Matrix.ix[averages_df.ix[index,'Node 1'], averages_df.ix[index,'Node 2']] = 1.0/counts_dict[averages_df.ix[index,'Node 2']]
for index, value in enumerate(averages_df.ix[:,'Negative']):
    if ~np.isnan(value):
        Matrix.ix[averages_df.ix[index,'Node 2'], averages_df.ix[index,'Node 1']] = 1.0/counts_dict[averages_df.ix[index,'Node 1']]

Matrix2= pd.DataFrame([1,1,1,1,1,1,1,1,1,1])

Matrix.fillna(0, inplace=True)
new_names = list(Matrix.index.values)
Matrix = np.matrix(Matrix) #Convert to Matrix
Matrix2 = np.matrix(Matrix2) #Convert to Matrix


#Find Eigenvector/PageRank
iterations = 25

for i in range(iterations):
    new = Matrix*Matrix2
    new = new/np.linalg.norm(new)
    if np.array_equal(np.array(x),np.array(new)):
        print 'Convergence after ', i
        break
    x = new



newseries = pd.DataFrame(new, index=new_names)

#Assign a index name to the pagerank
final = pd.concat([final, newseries], axis=1)
final.columns = ['Weighted PageRank','Sales','Unweighted PageRank']
print('Unwighted Correlation')
print(scipy.stats.spearmanr(final['Unweighted PageRank'], final['Sales']))
print final